In [32]:
import pandas as pd
import numpy as np

# Generate a dummy dataset with missing or empty values for specific columns
np.random.seed(42)

# Parameters for the dataset
num_rows = 50  # Number of rows to simulate
time_column = pd.date_range(start="2024-10-01 00:00:00", periods=num_rows, freq="5T")
instantaneous_values = np.random.uniform(1300, 1350, size=num_rows)
setpoint_values = np.random.choice([1350.0, None], size=num_rows, p=[0.8, 0.2])  # Add None for missing values
percent_open_values = np.random.choice([28.31, None], size=num_rows, p=[0.85, 0.15])  # Add None for missing values

# Create the DataFrame
dummy_data = pd.DataFrame({
    "Time": time_column,
    "Inj Gas Meter Volume Instantaneous": instantaneous_values,
    "Inj Gas Meter Volume Setpoint": setpoint_values,
    "Inj Gas Valve Percent Open": percent_open_values,
})

# Save to a CSV file
file_path = "dummy_dataset_with_missing_values.csv"
dummy_data.to_csv(file_path, index=False)
print(f"Dummy dataset saved as {file_path}")


Dummy dataset saved as dummy_dataset_with_missing_values.csv


/var/folders/hj/yvqd6fk50qbg72p7140xxl2w0000gn/T/ipykernel_16608/2463253466.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_column = pd.date_range(start="2024-10-01 00:00:00", periods=num_rows, freq="5T")


In [38]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the trained LSTM model
model_path = "project/lstm_model.h5"  # Replace with your saved model file path
lstm_model = load_model(model_path)

# Load the dummy dataset
dummy_file_path = "dummy_dataset_with_missing_values.csv"  # Replace with your dummy dataset file path
dummy_data = pd.read_csv(dummy_file_path)

# Handle missing values in the dummy dataset
def preprocess_dummy_data(df):
    # Fill missing values
    df['Inj Gas Meter Volume Setpoint'] = df['Inj Gas Meter Volume Setpoint'].fillna(method='ffill').fillna(method='bfill')
    df['Inj Gas Valve Percent Open'] = df['Inj Gas Valve Percent Open'].fillna(method='ffill').fillna(method='bfill')

    # Calculate derived features
    df['Volume Deviation'] = df['Inj Gas Meter Volume Setpoint'] - df['Inj Gas Meter Volume Instantaneous']
    epsilon = 1e-5  # Small value to prevent division by zero
    df['Valve Efficiency'] = df['Inj Gas Meter Volume Instantaneous'] / (df['Inj Gas Valve Percent Open'] + epsilon)

    # Drop rows with any remaining NaN values
    df = df.dropna()
    return df

# Preprocess the dummy data
processed_dummy_data = preprocess_dummy_data(dummy_data)

# Prepare data for prediction
X_dummy = processed_dummy_data[['Volume Deviation', 'Valve Efficiency']].values

# Reshape for LSTM input
timesteps = 1  # Match the timesteps used in training
features = X_dummy.shape[1]
X_dummy_lstm = X_dummy.reshape((X_dummy.shape[0], timesteps, features))

# Make predictions
dummy_predictions = lstm_model.predict(X_dummy_lstm)
dummy_predictions = (dummy_predictions > 0.5).astype(int)  # Convert probabilities to binary predictions

# Append predictions to the DataFrame
processed_dummy_data['Predictions'] = dummy_predictions

# Display results
print("Processed Dummy Data with Predictions:")
print(processed_dummy_data)

# Save results to a CSV file
processed_dummy_data.to_csv("project/dummy_dataset_with_missing_values.csv", index=False)
print("Predictions saved to 'dummy_dataset_with_predictions.csv'")


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'project/lstm_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)